In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,GridSearchCV
from xgboost import XGBClassifier
from collections import defaultdict

In [2]:
df = pd.read_csv('df_final.csv')

### 인구수 전처리

In [3]:
pop = pd.read_csv('행정구역_인구수.csv')

In [4]:
pop.head()
pop.인구수 = pop.인구수.apply(lambda x: x.replace(',',''))

In [5]:
pop.인구수 = pop.인구수.apply(lambda x: int(x))

In [6]:
len(pop.시)
pop_city = pop.시
pop_ppl = pop.인구수

pop_dict = defaultdict(int)

for city, num in zip(pop_city,pop_ppl):
    pop_dict[city] = int(num)

In [7]:
pop.head()

,대분류,시,인구수
0,충청남도,천안시,657821
1,충청남도,공주시,102651
2,충청남도,보령시,97188
3,충청남도,아산시,333602
4,충청남도,서산시,176440


In [8]:
pop_dict

defaultdict(int,
            {'천안시': 657821,
             '공주시': 102651,
             '보령시': 97188,
             '아산시': 333602,
             '서산시': 176440,
             '논산시': 112842,
             '계룡시': 44328,
             '당진시': 167955,
             '금산군': 50092,
             '부여군': 62435,
             '서천군': 50009,
             '청양군': 30270,
             '홍성군': 98053,
             '예산군': 77329,
             '태안군': 61342,
             '조치원읍': 42408,
             '연기면': 2653,
             '연동면': 2996,
             '부강면': 5900,
             '금남면': 8699,
             '장군면': 6898,
             '연서면': 7261,
             '전의면': 5493,
             '전동면': 3288,
             '소정면': 2219,
             '한솔동': 18335,
             '새롬동': 39636,
             '도담동': 35356,
             '해밀동': 8704,
             '아름동': 23444,
             '종촌동': 28664,
             '고운동': 35076,
             '소담동': 21461,
             '반곡동': 25644,
             '보람동': 18941,
             '대평동': 11262,
             '

### 인구수 ->df 합치기

In [9]:
df.중분류.value_counts()

천안시    17784
서구     12121
아산시     8309
유성구     8102
중구      8101
       ...  
음성군        6
충주시        5
제천시        3
보은군        1
증평군        1
Name: 중분류, Length: 62, dtype: int64

In [10]:
# 인구수 있는 것 채우기
df['중분류_인구수'] = df['중분류'].apply(lambda x: pop_dict[x] if x in pop_dict else None)

In [11]:
df.head()

,Unnamed: 0,시간,사건종별,주소,x,y,행정구역,대분류,중분류,요일,hour,시간대,주말,공휴일,휴일,음주운전 여부,중분류_인구수
0,0,2020-12-01 0:00,음주운전,세종특별자치시 세종시 소담동(소담동) 286-3,127.298872,36.489101,세종특별자치시 소담동,세종특별자치시,소담동,Tuesday,0,새벽,0,0,0,1,21461.0
1,1,2020-12-01 0:00,음주운전,충청남도 아산시 둔포면 석곡리(둔포면) 1371,127.058056,36.923056,충청남도 아산시 둔포면 석곡리,충청남도,아산시,Tuesday,0,새벽,0,0,0,1,333602.0
2,2,2020-12-01 0:00,교통사고,대전광역시 서구 괴정동(행정:괴정동) 95-28,127.382337,36.336437,대전광역시 서구 괴정동,대전광역시,서구,Tuesday,0,새벽,0,0,0,0,470374.0
3,3,2020-12-01 0:00,음주운전,충청남도 천안시 동남구 신방동(신방동) 920,127.128056,36.787778,충청남도 천안시 동남구 신방동,충청남도,천안시,Tuesday,0,새벽,0,0,0,1,657821.0
4,4,2020-12-01 0:00,음주운전,충청남도 아산시 배방읍 공수리(행정:배방읍) 141-7,127.057063,36.774519,충청남도 아산시 배방읍 공수리,충청남도,아산시,Tuesday,0,새벽,0,0,0,1,333602.0


In [12]:
df['중분류_인구수'].value_counts() #None 없음 확인

657821.0    17784
470374.0    12121
333602.0     8309
356093.0     8102
227108.0     8101
219751.0     5611
167955.0     5101
172746.0     5034
176440.0     4439
102651.0     3789
112842.0     3077
97188.0      3028
98053.0      2320
77329.0      1849
61342.0      1564
50009.0      1011
62435.0      1003
42408.0       982
50092.0       971
44328.0       485
30270.0       459
8699.0        410
28664.0       285
23444.0       276
35076.0       274
28432.0       270
6898.0        247
35356.0       216
11262.0       204
849573.0      178
18941.0       174
21461.0       160
39636.0       146
5900.0        137
2219.0        132
25644.0       130
7261.0        123
18335.0       115
5493.0        115
2653.0         94
3288.0         83
2996.0         80
8704.0         53
49520.0        34
86147.0        15
44956.0         7
92058.0         6
208277.0        5
130988.0        3
31455.0         1
37262.0         1
Name: 중분류_인구수, dtype: int64

### 음주운전 이진변수로 바꾸기

In [13]:
df['사건종별']

0         음주운전
1         음주운전
2         교통사고
3         음주운전
4         음주운전
          ... 
100320    교통위반
100321    교통사고
100322    교통사고
100323    음주운전
100324    음주운전
Name: 사건종별, Length: 100325, dtype: object

In [14]:
#사건 column이진분류로 만들기
df['사건'] = df['사건종별'].apply(lambda x: int(x == '음주운전'))

In [15]:
df.head()

,Unnamed: 0,시간,사건종별,주소,x,y,행정구역,대분류,중분류,요일,hour,시간대,주말,공휴일,휴일,음주운전 여부,중분류_인구수,사건
0,0,2020-12-01 0:00,음주운전,세종특별자치시 세종시 소담동(소담동) 286-3,127.298872,36.489101,세종특별자치시 소담동,세종특별자치시,소담동,Tuesday,0,새벽,0,0,0,1,21461.0,1
1,1,2020-12-01 0:00,음주운전,충청남도 아산시 둔포면 석곡리(둔포면) 1371,127.058056,36.923056,충청남도 아산시 둔포면 석곡리,충청남도,아산시,Tuesday,0,새벽,0,0,0,1,333602.0,1
2,2,2020-12-01 0:00,교통사고,대전광역시 서구 괴정동(행정:괴정동) 95-28,127.382337,36.336437,대전광역시 서구 괴정동,대전광역시,서구,Tuesday,0,새벽,0,0,0,0,470374.0,0
3,3,2020-12-01 0:00,음주운전,충청남도 천안시 동남구 신방동(신방동) 920,127.128056,36.787778,충청남도 천안시 동남구 신방동,충청남도,천안시,Tuesday,0,새벽,0,0,0,1,657821.0,1
4,4,2020-12-01 0:00,음주운전,충청남도 아산시 배방읍 공수리(행정:배방읍) 141-7,127.057063,36.774519,충청남도 아산시 배방읍 공수리,충청남도,아산시,Tuesday,0,새벽,0,0,0,1,333602.0,1


In [17]:
df.to_csv('df_final_인구수.csv',encoding = 'utf-8-sig')

## x data 전처리
xgboost에 넣으려면 text data는 모두 one-hot encoding을 하거나 해야함

In [18]:
df.columns

Index(['Unnamed: 0', '시간', '사건종별', '주소', 'x', 'y', '행정구역', '대분류', '중분류', '요일',
       'hour', '시간대', '주말', '공휴일', '휴일', '음주운전 여부', '중분류_인구수', '사건'],
      dtype='object')

In [19]:
df['중분류'].value_counts()

천안시    17784
서구     12121
아산시     8309
유성구     8102
중구      8101
       ...  
음성군        6
충주시        5
제천시        3
보은군        1
증평군        1
Name: 중분류, Length: 62, dtype: int64

In [20]:
# 요일, 시간대, 휴일, 
#행정구역(중분류)

#못모은거: 날씨, 도로길이, 속도, 교통량, 행정규역별 차량대수
# 중분류를 일단은 빼겠음. 차원의 저주가 생길 것 같아서... 
x_raw = df[['시간대','휴일','요일','중분류_인구수']] #여기에 추가될 것. 
y = df['사건']

In [21]:
x_raw

,시간대,휴일,요일,중분류_인구수
0,새벽,0,Tuesday,21461.0
1,새벽,0,Tuesday,333602.0
2,새벽,0,Tuesday,470374.0
3,새벽,0,Tuesday,657821.0
4,새벽,0,Tuesday,333602.0
...,...,...,...,...
100320,퇴근,1,Saturday,8699.0
100321,퇴근,1,Saturday,470374.0
100322,낮,0,Friday,470374.0
100323,낮,0,Friday,657821.0


In [22]:
# Select the qualitative data column

qualitatives = ['시간대','요일']

for col in qualitatives:
    qual_data = x_raw[col]

    # Use the get_dummies method to perform one-hot encoding
    one_hot_encoded = pd.get_dummies(qual_data)
        
    # Concatenate the one-hot encoded data with the original data
    encoded_data = pd.concat([x_raw, one_hot_encoded], axis=1)

    # Drop the original qualitative data column
    encoded_data = encoded_data.drop(col, axis=1)

In [23]:
qual_data = x_raw['시간대']

# Use the get_dummies method to perform one-hot encoding
one_hot_encoded = pd.get_dummies(qual_data)

# Concatenate the one-hot encoded data with the original data
encoded_data = pd.concat([encoded_data, one_hot_encoded], axis=1)

# Drop the original qualitative data column
encoded_data = encoded_data.drop('시간대', axis=1)

In [24]:
encoded_data

,휴일,중분류_인구수,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,낮,밤,새벽,아침,저녁,출근,퇴근
0,0,21461.0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
1,0,333602.0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
2,0,470374.0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
3,0,657821.0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
4,0,333602.0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100320,1,8699.0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
100321,1,470374.0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
100322,0,470374.0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
100323,0,657821.0,1,0,0,0,0,0,0,1,0,0,0,0,0,0


In [115]:
x = encoded_data

In [116]:
# 모델 학습 

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

# Define the model
xgb_model = XGBClassifier()

# Define the parameter grid
param_grid = {
    'max_depth': [3, 5, 7],
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.1, 0.2, 0.3],
    'gamma': [0, 0.1, 0.2],
    'subsample': [0.5, 0.8, 1.0],
    'colsample_bytree': [0.5, 0.8, 1.0]
}

# Define the GridSearchCV object
grid_search = GridSearchCV(xgb_model, param_grid, scoring='accuracy', cv=3, n_jobs=-1)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train, y_train)

# Predict the target values for the test data using the best model
y_pred = grid_search.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(accuracy * 100))

KeyboardInterrupt: 

In [25]:
encoded_data = pd.concat([encoded_data,y], axis=1)

In [26]:
encoded_data.to_csv('encod.csv', encoding = 'utf-8-sig')

In [27]:
encoded_data

,휴일,중분류_인구수,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,낮,밤,새벽,아침,저녁,출근,퇴근,사건
0,0,21461.0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1
1,0,333602.0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1
2,0,470374.0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
3,0,657821.0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1
4,0,333602.0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100320,1,8699.0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
100321,1,470374.0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
100322,0,470374.0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0
100323,0,657821.0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1


In [135]:
df

,Unnamed: 0,시간,사건종별,주소,x,y,행정구역,대분류,중분류,요일,hour,시간대,주말,공휴일,휴일,음주운전 여부,중분류_인구수,사건
0,0,2020-12-01 0:00,음주운전,세종특별자치시 세종시 소담동(소담동) 286-3,127.298872,36.489101,세종특별자치시 소담동,세종특별자치시,소담동,Tuesday,0,새벽,0,0,0,1,21461.0,1
1,1,2020-12-01 0:00,음주운전,충청남도 아산시 둔포면 석곡리(둔포면) 1371,127.058056,36.923056,충청남도 아산시 둔포면 석곡리,충청남도,아산시,Tuesday,0,새벽,0,0,0,1,333602.0,1
2,2,2020-12-01 0:00,교통사고,대전광역시 서구 괴정동(행정:괴정동) 95-28,127.382337,36.336437,대전광역시 서구 괴정동,대전광역시,서구,Tuesday,0,새벽,0,0,0,0,470374.0,0
3,3,2020-12-01 0:00,음주운전,충청남도 천안시 동남구 신방동(신방동) 920,127.128056,36.787778,충청남도 천안시 동남구 신방동,충청남도,천안시,Tuesday,0,새벽,0,0,0,1,657821.0,1
4,4,2020-12-01 0:00,음주운전,충청남도 아산시 배방읍 공수리(행정:배방읍) 141-7,127.057063,36.774519,충청남도 아산시 배방읍 공수리,충청남도,아산시,Tuesday,0,새벽,0,0,0,1,333602.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100320,100320,2022-02-19 17:00,교통위반,세종특별자치시 세종시 금남면 국곡리(금남면 ) 산4-2,127.273611,36.435000,세종특별자치시 금남면 국곡리,세종특별자치시,금남면,Saturday,17,퇴근,1,0,1,0,8699.0,0
100321,100321,2022-02-19 17:00,교통사고,대전 서구 둔산동 1063,127.376396,36.350271,대전광역시 서구 둔산동,대전광역시,서구,Saturday,17,퇴근,1,0,1,0,470374.0,0
100322,100322,2022-07-01 13:00,교통사고,대전광역시 서구 정림동 (행정:정림동 ) 8-56,127.370828,36.309233,대전광역시 서구 정림동,대전광역시,서구,Friday,13,낮,0,0,0,0,470374.0,0
100323,100323,2022-07-01 14:00,음주운전,충청남도 천안시 동남구 목천읍 신계리(목천읍 ) 89-3,127.221604,36.773741,충청남도 천안시 동남구 목천읍 신계리,충청남도,천안시,Friday,14,낮,0,0,0,1,657821.0,1
